In [ ]:
!pip install geojson
!pip install rasterio

In [ ]:
import os
import gdal
from matplotlib import pyplot as plt
import numpy as np
import cv2
import pandas as pd
import rasterio
from rasterio.mask import mask
import geojson
import random
import argparse
from osgeo import gdal
from osgeo.gdalconst import GDT_Float32
import sys

from sklearn.base import clone
from sklearn import metrics
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
def get_img(snapshot):
    plt.figure(figsize=(7, 7))
    plt.axis("off")
    plt.imshow(cv2.merge([snapshot[3], snapshot[2], snapshot[1]]))

In [ ]:
def create_df(field_g, field_class, max_canal, num_def_field):
    df_def_g = pd.DataFrame(columns=[i for i in range(max_canal)]+['class', 'id'])
    for field_i in field_g:
        df_def_i = pd.DataFrame(field_i.reshape(max_canal, -1).transpose())
        df_def_i['id'] = num_def_field
        num_def_field += 1
        #df_def_i = df_def_i.loc[(df_def_i[2]>0)&(df_def_i[3]>0)]
        df_def_g = pd.concat([df_def_g, df_def_i], ignore_index=True)
    df_def_g['class'] = field_class
    return df_def_g, num_def_field

In [ ]:
def get_id_test(df_def, part_test, max_shuffle=30):
    df_def_count = df_def.groupby('id', as_index=False).count()[['id', 'class']]
    max_count = df_def_count['class'].sum()
    min_count = 1
    min_id_test = []
    i_def = 0
    while i_def<max_shuffle and abs(min_count-part_test)>0.01:
        sum_count = 0
        id_test = []
        for index_row, df_count_row in df_def_count.sample(frac=1).iterrows():
            sum_count += df_count_row['class']
            id_test.append(df_count_row['id'])
            cur_count = sum_count/max_count
            if cur_count>=part_test-0.02:
                if abs(cur_count-part_test)<abs(min_count-part_test):
                    min_count = cur_count
                    min_id_test = id_test
                i_def += 1
                break
    if min_count>part_test*2 or part_test-0.96>0:
        min_id_test = []
    return min_id_test

In [ ]:
def create_prepared_df(list_df_def, list_data_def, remove_cloud=True, with_class=False):
    df_def = pd.DataFrame()
    for data_def, df_i_def in zip(list_data_def, list_df_def):
        df_i_def = df_i_def.rename(columns={x:str(x)+'_'+data_def for x in (df_i_def.columns[:-2] if with_class else df_i_def.columns)})
        df_def = df_def.join(df_i_def[(df_i_def.columns[:-2] if with_class else df_i_def.columns)], how="outer")
    if with_class:
        df_def['class'] = df_i_def['class']
    if remove_cloud:
        for col in df_def.columns[:-1]:
            if col.split('_')[0]=='12':
                df_def = df_def.loc[df_def[col]<1].reset_index(drop=True)
    return df_def

In [ ]:
def kill_max_class(df_def):
    df_count = df_def.groupby('class', as_index=False).count()
    df_count.index = df_count['class']
    class_median = int(df_count[df_count.columns[1]].median())
    max_class = df_count[df_count.columns[1]].idxmax()
    df_def_max = df_def.loc[df_def['class']==max_class]
    df_def_max['ind'] = df_def_max.index
    df_def_max = df_def_max.loc[df_def_max['ind'].isin(random.sample(list(df_def_max.index), class_median))]
    df_def = pd.concat([df_def.loc[df_def['class']!=max_class], df_def_max[df_def_max.columns[:-1]]], ignore_index=True).sort_values(['class']).reset_index(drop=True)
    return df_def

In [ ]:
def write_geotiff(raster_input, raster_output, predictions):
    """
    Function to replace values in geotiff upper then threshold by thresholf  🙂
    
    Input : GeoTIFF, threshold_value
    
    Output : GeoTIFF 
    
    """    
    in_data, out_data = None, None

    # open input raster
    in_data = gdal.Open(raster_input)
    if in_data is None:
        print ('Unable to open %s' % raster_input)
        return None

    # read in data from first band of input raster
    band1 = in_data.GetRasterBand(1)
    rows = in_data.RasterYSize
    cols = in_data.RasterXSize
    vals = band1.ReadAsArray(0, 0, cols, rows)

    driver = in_data.GetDriver()
    out_data = driver.Create(raster_output, cols, rows, 1, GDT_Float32)

    dem_data = np.array(predictions)
    out_band = out_data.GetRasterBand(1)
    out_band.WriteArray(dem_data)
    out_band.FlushCache()
    out_band.SetNoDataValue(-32767.)

    out_data.SetGeoTransform(in_data.GetGeoTransform())
    out_data.SetProjection(in_data.GetProjection())
    del out_data
    return raster_output

In [ ]:
path = "Forest/data/"
name_field = 'Bratsk'
resolution = '10'
folder_out = '10_new'
#folder_out = '10_only_forest_new'

path_field = path + name_field + '/' + resolution + '/'
path_field_year = [path_field+f+'/' for f in os.listdir(path_field) if f!='Masks']
path_snapshots = [field_year+snapshot for field_year in path_field_year for snapshot in os.listdir(field_year)]
path_masks = path + name_field + '/' + 'Masks/In/' + os.listdir(path + name_field + '/' + 'Masks/In/')[0]
path_out_masks = path + name_field + '/' + 'Masks/Out/' + folder_out + '/'

if not os.path.exists(path_out_masks):
    os.mkdir(path_out_masks)

In [ ]:
remove_autumn_months = True
if remove_autumn_months:
    while [path_snapshot for path_snapshot in path_snapshots if (int(path_snapshot.split('/')[-1].split('.')[0][6:7])<6 or int(path_snapshot.split('/')[-1].split('.')[0][6:7])>8)]:
        for path_snapshot in [path_snapshot for path_snapshot in path_snapshots if (int(path_snapshot.split('/')[-1].split('.')[0][6:7])<6 or int(path_snapshot.split('/')[-1].split('.')[0][6:7])>8)]:
            path_snapshots.remove(path_snapshot)

In [ ]:
count_cloud = float("inf")
for path_snapshot in path_snapshots:
    if path_snapshot[-9]=='7' or path_snapshot[-9]=='8':
        #print(path_snapshot)
        field = np.array(gdal.Open(path_snapshot).ReadAsArray())
        df_i = pd.DataFrame(field.reshape(13, -1).transpose())
        if len(df_i.loc[df_i[12]>0])<count_cloud:
            count_cloud = len(df_i.loc[df_i[12]>0])
            path_min_cloud = path_snapshot
        if count_cloud==0:
            print(path_snapshot)
            #break

In [ ]:
field = np.array(gdal.Open(path_min_cloud).ReadAsArray())
get_img(field)

with open(path_masks) as f:
    gj = geojson.load(f)
features = [feature["geometry"] for feature in gj['features']]

with rasterio.open(path_min_cloud) as src:
    out_image, _ = mask(src, features, crop=True)

get_img(out_image)

In [ ]:
def convert_to_geotiff(raster_input, raster_output, values_to_write):
    """
    Save GPP resul geotiff GeoTiff

    input: path to geotiff with fPAR 
    output: path to new geotiff with GPP 
    values_to_write: np.array with dimensions like input geotiff file

    """
    in_data, out_data = None, None
    in_data = gdal.Open(raster_input)
    if in_data is None:
        print ('Unable to open %s' % raster_input)

    # read in data from first band of input raster
    band1 = in_data.GetRasterBand(1)
    rows = in_data.RasterYSize
    cols = in_data.RasterXSize
    vals = band1.ReadAsArray(0, 0, cols, rows)

    driver = in_data.GetDriver()

    num_of_layers = values_to_write.shape[0]

    out_data = driver.Create(raster_output, cols, rows, num_of_layers, GDT_Float32)

    # Save data
    for i in range(num_of_layers):
        out_band = out_data.GetRasterBand(i+1)
        out_band.WriteArray(values_to_write[i, :, :])
    out_band.FlushCache()
    out_band.SetNoDataValue(-32767.)

    out_data.SetGeoTransform(in_data.GetGeoTransform())
    out_data.SetProjection(in_data.GetProjection())
    del out_data
    return raster_output

In [ ]:
path_Haralick = '/Forest/HaralickTask_all_part_improve/'
path_Haralick_features = [path_Haralick+f+'/' for f in os.listdir(path_Haralick) if f!='tiff']
path_Haralick_npy = 'data/BANDS-S2-L2A_HARALICK.npy'

In [ ]:
path_terrain = '/Forest/Terrain/'

In [ ]:
input_raster = path_min_cloud
output_raster = path_Haralick+'tiff/'
if not os.path.exists(output_raster):
    os.mkdir(output_raster)

field = np.array(gdal.Open(path_min_cloud).ReadAsArray())
Haralick_npy_shape = np.load(path_Haralick_features[0]+'0/'+path_Haralick_npy)[0]
for path_Haralick_feature in path_Haralick_features:
    if not os.path.exists(output_raster+path_Haralick_feature.split(path_Haralick)[1].split('/')[0]+'.tiff'):
        array_reloaded = np.zeros(shape=(field.shape[1], field.shape[2], Haralick_npy_shape.shape[2]))
        for i in range(4):
            array_reloaded[Haralick_npy_shape.shape[0]*(1-i%2):Haralick_npy_shape.shape[0]*(2-i%2),Haralick_npy_shape.shape[1]*(i//2):Haralick_npy_shape.shape[1]*(i//2+1),:] = np.load(path_Haralick_feature+str(i)+'/'+path_Haralick_npy)[0]
        res_fname = convert_to_geotiff(raster_input = input_raster, raster_output=output_raster+path_Haralick_feature.split(path_Haralick)[1].split('/')[0]+'.tiff', values_to_write=array_reloaded.transpose(2, 0, 1))
        print(res_fname)

In [ ]:
path_Haralick_tiffs = [output_raster+f for f in os.listdir(output_raster)]

In [ ]:
def generate_NDVI(features):
    nir = features[7]
    red = features[3]
    ndvi = (nir-red)/((nir+red).apply(lambda x: 0.000000001 if x==0 else x))
    return ndvi

In [ ]:
def generate_EVI(features):
    evi2 = 2.5*(features[7] - features[3])/((features[7]  + 2.4*features[3] + 1).apply(lambda x: 0.000000001 if x==0 else x))
    return evi2

In [ ]:
def generate_NDRE(features):
    ndre = (features[7] - features[4])/((features[7] + features[4]).apply(lambda x: 0.000000001 if x==0 else x))
    return ndre

In [ ]:
def generate_MSAVI(features):
    nir = features[7]
    red = features[3]
    msavi=(2*nir + 1 - ((2*nir+1)**2 - 8*(nir-red))**(1/2))/2
    return msavi

In [ ]:
def generate_all_indices(df_def):
    df_def_col = list(df_def.columns)
    df_def['ndvi'] = generate_NDVI(df_def)
    df_def['evi'] = generate_EVI(df_def)
    df_def['ndre'] = generate_NDRE(df_def)
    df_def['msavi'] = generate_MSAVI(df_def)
    df_def = df_def.reindex(columns=df_def_col[:-2]+list(df_def.columns[-4:])+df_def_col[-2:])
    return df_def

In [ ]:
def del_row(df_def, create_id_test=False):
    del_row_col = []
    for col in df_def.columns[:-1]:
        if not create_id_test:
            if (col[0]=='2' or col[0]=='3') and col[2]=='2':
                del_row_col.append(col)
        else:
            if str(col)[0]=='2' or str(col)[0]=='3':
                del_row_col.append(col)
    del_row_cond = (df_def[del_row_col[0]]>0)
    for col in del_row_col[1:]:
        del_row_cond = del_row_cond&(df_def[col]>0)

    df_def = df_def.loc[del_row_cond].reset_index(drop=True)
    return df_def

In [ ]:
list_df = []
list_df_test = []
list_data = []
dict_field_id = {}

part_test = 0.3
max_snapshot = 10000
max_shuffle = 30
flag_remove_cloud = False
create_only_field = False
kill_class = True

max_class = float("inf")
if create_only_field:
    max_class = 7

with open(path_masks) as f:
    gj = geojson.load(f)
features_masks = [feature["geometry"] for feature in gj['features']]
features_desc = [[feature["properties"]["t_Class"], feature["properties"]["t_Клас"]] for feature in gj['features']]
features_ind = [feature["properties"]["t_Class"] for feature in gj['features']]

dict_desc = {}
for key_i, i in list(set(tuple(row) for row in features_desc)):
    dict_desc[key_i] = i

for snapshot_i, path_snapshot in enumerate(path_snapshots+path_Haralick_tiffs+path_terrain):
    if snapshot_i<max_snapshot:
        print(path_snapshot)
        field = np.array(gdal.Open(path_snapshot).ReadAsArray())

        list_field = []
        for feature in features_masks:
            with rasterio.open(path_snapshot) as src:
                out_image, out_transform = mask(src, [feature], crop=True)
                list_field.append(out_image)
        
        list_field_ind_g = []
        features_ind_g = []
        for i in range(min(features_ind), max(features_ind)+1):
            indices_i = [feature_ind for feature_ind, x in enumerate(features_ind) if x == i]
            if len(indices_i)>0:
                list_field_ind_g.append([list_field[index] for index in indices_i])
                features_ind_g.append(i)
        
        max_canal = list_field[0].shape[0]
        
        num_field = 0
        df = pd.DataFrame(columns=[i for i in range(max_canal)]+['class'])
        df_test = pd.DataFrame(columns=[i for i in range(max_canal)]+['class'])

        for ind_g, field_g_i in zip(features_ind_g, list_field_ind_g):
            df_g_i, num_field = create_df(field_g_i, ind_g, max_canal, num_field)
            if snapshot_i==0:
                df_g_i_0 = df_g_i.copy()
                df_g_i_0 = del_row(df_g_i_0, create_id_test=True)
                id_field = get_id_test(df_g_i_0, part_test, max_shuffle)
                dict_field_id[ind_g] = id_field
            else:
                id_field = dict_field_id[ind_g]
            if id_field:
                df = pd.concat([df, df_g_i.loc[~df_g_i['id'].isin(id_field)]], ignore_index=True)
                df_test = pd.concat([df_test, df_g_i.loc[df_g_i['id'].isin(id_field)]], ignore_index=True)
            else:
                for num_field_i in df_g_i['id'].unique():
                    df_i = df_g_i.loc[df_g_i['id']==num_field_i].reset_index(drop=True)
                    test_ind = int(round(len(df_i)*part_test))
                    df = pd.concat([df, df_i.loc[df_i.index[:-test_ind]]], ignore_index=True)
                    df_test = pd.concat([df_test, df_i.loc[df_i.index[-test_ind:]]], ignore_index=True)
        
        if snapshot_i<len(path_snapshots):
            df = generate_all_indices(df.copy())
            df_test = generate_all_indices(df_test.copy())
        
        list_df.append(df)
        list_df_test.append(df_test)
        list_data.append(path_snapshot.split('/')[-1].split('.')[0])

for list_i in [list_df, list_df_test]:
    dev_df = []
    for df_i in list_i:
        dev_df.append(len(df_i))
    if len(set(dev_df))>1:
        print('Разная длина у DataFrame')

df = create_prepared_df(list_df, list_data, remove_cloud=flag_remove_cloud, with_class=True)
df_test = create_prepared_df(list_df_test, list_data, remove_cloud=flag_remove_cloud, with_class=True)

df = del_row(df)
df_test = del_row(df_test)

if kill_class:
    df = kill_max_class(df)
    df_test = kill_max_class(df_test)

for col in df.columns:
    df[col] = pd.to_numeric(df[col])
for col in df_test.columns:
    df_test[col] = pd.to_numeric(df_test[col])

if create_only_field:
    df = df.loc[df['class']<=max_class].reset_index(drop=True)
    df_test = df_test.loc[df_test['class']<=max_class].reset_index(drop=True)

In [ ]:
# norm image
norm_image = False
if norm_image:
    for col in df.columns:
        if col.split('_')[0] not in ['ndvi', 'evi', 'ndre', 'msavi', 'class', '12']:
            df[col] = (df[col]-df[col].mean())/df[col].std()
            df_test[col] = (df_test[col]-df_test[col].mean())/df_test[col].std()

In [ ]:
for col in df.columns[:-1]:
    if col.split('_')[0] in ['1','2','3','7','8','12']:
      del df[col]
for col in df_test.columns[:-1]:
    if col.split('_')[0] in ['1','2','3','7','8','12']:
      del df_test[col]

In [ ]:
X_train = df[df.columns[:-1]]
y_train = list(df['class'])
X_test = df_test[df_test.columns[:-1]]
y_test = list(df_test['class'])

In [ ]:

base_classfiers = [KNeighborsClassifier(n_jobs=-1, algorithm='ball_tree', leaf_size=100, n_neighbors=10, weights='uniform'),
                   DecisionTreeClassifier(random_state=42, criterion='entropy', max_depth=9, max_features=None, min_samples_leaf=3, min_samples_split=2, splitter='best'),
                   RandomForestClassifier(n_jobs=-1, random_state=42, criterion='gini', max_features='auto', class_weight=None, max_depth=50, n_estimators=500, min_samples_leaf=2, min_samples_split=6),
                   ExtraTreesClassifier(n_jobs=-1, random_state=42, criterion='entropy', max_depth=9, max_features='log2', min_samples_leaf=5, min_samples_split=2, n_estimators=150),
                   RidgeClassifier(random_state=42, solver='sag', class_weight=None, fit_intercept=True, normalize=True, alpha=1.1, tol=1e-5),
                   LogisticRegression(n_jobs=-1, random_state=42, class_weight='balanced', dual=False, fit_intercept=False, C=1.2, max_iter=100, tol=1e-04, penalty='l1', solver='saga'),
                   SVC(random_state=42, gamma='scale', kernel='poly', C=1, degree=1, tol=1e-5, probability=True),
                   XGBClassifier(n_jobs=-1, tree_method='gpu_hist', predictor='gpu_predictor', booster='gblinear', eta=0.3, gamma='auto', max_depth=20)
                   ]
name_classfiers = ['KNeighborsClassifier',
                   'DecisionTreeClassifier',
                   'RandomForest',
                   'ExtraTreesClassifier',
                   'RidgeClassifier',
                   'LogisticRegression',
                   'SVC',
                   'XGBClassifier'
                   ]

for i in range(len(base_classfiers)):
    base_classfiers[i].fit(X_train, y_train)
    print('Done: '+name_classfiers[i])

In [ ]:
score_classfiers_accuracy_score = []
score_classfiers_roc_auc_score = []
score_classfiers_f1_score = []
df_score_class_list = []
for i in range(len(base_classfiers)):
    y_predict = base_classfiers[i].predict(X_test)
    score_classfiers_accuracy_score.append(accuracy_score(y_test, y_predict))
    if name_classfiers[i]!='RidgeClassifier':
        score_classfiers_roc_auc_score.append(roc_auc_score(y_test, base_classfiers[i].predict_proba(X_test), multi_class='ovr'))
    else:
        ridge_predict = []
        for k in range(len(X_test)):
            d = base_classfiers[i].decision_function(X_test)[k]
            probs = np.exp(d) / np.sum(np.exp(d))
            ridge_predict.append(probs)
        ridge_predict = np.array(ridge_predict)
        score_classfiers_roc_auc_score.append(roc_auc_score(y_test, ridge_predict, multi_class='ovr')) 

    score_classfiers_f1_score.append(f1_score(y_test, y_predict, average='weighted'))
    df_i = pd.DataFrame(metrics.classification_report(y_test, y_predict, digits=2, output_dict=True)).transpose()
    arrays_col = [
        [name_classfiers[i], name_classfiers[i], name_classfiers[i], name_classfiers[i]],
        list(df_i.columns),
    ]
    df_i.columns = pd.MultiIndex.from_tuples(list(zip(*arrays_col)))
    df_score_class_list.append(df_i)

df_score_class = df_score_class_list[0]
for i in range(1,len(df_score_class_list)):
    df_score_class = df_score_class.join(df_score_class_list[i])
df_score_class_index = list(df_score_class.index)
for i, ind in enumerate(df_score_class_index):
    if ind.isdigit():
        df_score_class_index[i] = dict_desc[int(ind)]
df_score_class.index = df_score_class_index

df_score_group = pd.DataFrame(columns=['Model', 'Accuracy score', 'ROC AUC score', 'f1 score'])
df_score_group['Model'] = name_classfiers
df_score_group['Accuracy score'] = score_classfiers_accuracy_score
df_score_group['ROC AUC score'] = score_classfiers_roc_auc_score
df_score_group['f1 score'] = score_classfiers_f1_score

#df_score_group.to_excel(path_out_masks+'score_by_model_roc.xlsx', index=False)
#df_score_class.to_excel(path_out_masks+'score_by_group_roc.xlsx')

In [ ]:
list_df_fi = []
list_data_fi = []
for snapshot_i, path_snapshot in enumerate(path_snapshots+path_Haralick_tiffs):
    if snapshot_i<max_snapshot:
        print(path_snapshot)
        field = np.array(gdal.Open(path_snapshot).ReadAsArray())
        max_canal = field.shape[0]
        df_i = pd.DataFrame(field.reshape(max_canal, -1).transpose())
        list_df_fi.append(df_i)
        list_data_fi.append(path_snapshot.split('/')[-1].split('.')[0])
        if snapshot_i == 0:
            field_shape = field.shape

dev_df = []
for df_i in list_df_fi:
    dev_df.append(len(df_i))
if len(set(dev_df))>1:
    print('Разная длина у DataFrame')

#df_res = create_prepared_df(list_df_fi, list_data_fi, remove_cloud=False, with_class=False)

#del list_df_fi
#del df_i

#list_df_res = []
#for i in range(int(len(df_res)/5000)+1):
#    list_df_res.append(df_res[i*5000:(i+1)*5000])

list_df_res = []
len_df_fi = len(list_df_fi[0])
for i in range(int(len_df_fi/500000)+1):
    list_df_res.append(pd.DataFrame())
for data_i, df_i in zip(list_data_fi, list_df_fi):
    df_i = df_i.rename(columns={x:str(x)+'_'+data_i for x in df_i.columns})
    for i in range(int(len_df_fi/500000)+1):
        list_df_res[i] = list_df_res[i].join(df_i[i*500000:(i+1)*500000], how="outer")

del list_df_fi
del df_i

for i in range(len(base_classfiers)):
    res = np.empty([0], dtype=int)
    for df_res_i in list_df_res:
        res = np.concatenate((res, base_classfiers[i].predict(df_res_i)))
    print('Подготовка к сохранению')
    outPut = write_geotiff(path_min_cloud, path_out_masks+name_classfiers[i]+'_predictions_har.tif', res.reshape(field_shape[1:]))
    print('Done: '+name_classfiers[i])